## Qiskit implementation of a toy-cipher

### First we define the modulo addition circuit in ripple carry aproach [vide. https://doi.org/10.26421/QIC10.9-10-12]

In [5]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit_aer import AerSimulator

# Read user inputs
bitsize = int(input("Enter bit size: "))
num1    = int(input("Enter first number (0 to {}): ".format(2**bitsize-1)))
num2    = int(input("Enter second number (0 to {}): ".format(2**bitsize-1)))

# Validate
if not (0 <= num1 < 2**bitsize and 0 <= num2 < 2**bitsize):
    raise ValueError(f"Numbers must be in range [0, {2**bitsize-1}] for {bitsize}-bit registers.")

# Helper: ripple-carry adder
def modulo_addition(qc, qra, qrb):
    # Step 1
    for i in range(1, bitsize):
        qc.cx(qra[i], qrb[i])
    # Step 2
    for i in range(bitsize-2, 0, -1):
        qc.cx(qra[i], qra[i+1])
    # Step 3
    for i in range(bitsize-1):
        qc.ccx(qra[i], qrb[i], qra[i+1])
    # Step 4
    for i in range(bitsize-1, 0, -1):
        qc.cx(qra[i], qrb[i])
        qc.ccx(qra[i-1], qrb[i-1], qra[i])
    # Step 5
    for i in range(1, bitsize-1):
        qc.cx(qra[i], qra[i+1])
    # Step 6
    for i in range(bitsize):
        qc.cx(qra[i], qrb[i])

# Build circuit
qra = QuantumRegister(bitsize, 'a')
qrb = QuantumRegister(bitsize, 'b')
cr  = ClassicalRegister(bitsize, 'result')
qc  = QuantumCircuit(qra, qrb, cr)

# Initialize registers (LSB at index 0)
s1 = format(num1, f"0{bitsize}b")[::-1]
s2 = format(num2, f"0{bitsize}b")[::-1]
for i, bit in enumerate(s1):
    if bit == '1':
        qc.x(qra[i])
for i, bit in enumerate(s2):
    if bit == '1':
        qc.x(qrb[i])

# Perform addition
modulo_addition(qc, qra, qrb)

# Measure result register
qc.measure(qrb, cr)

# Simulate
aer = AerSimulator()
job = aer.run(qc.decompose(), shots=1)
res = job.result()
counts = res.get_counts()
# Extract most probable
output_bin = max(counts, key=counts.get)
output_int = int(output_bin, 2)

print(f"\nInput:  {num1} + {num2} (mod 2^{bitsize})")
print(f"Output (binary MSB→LSB): {output_bin}")
print(f"Output (decimal)   : {output_int}")


Input:  14 + 10 (mod 2^4)
Output (binary MSB→LSB): 1000
Output (decimal)   : 8


### Now let us define the toy cipher

In [ ]:
import random

bitsize = 4  # 4-bit registers

# --- rotate-left by n (mod 4) on a 4-qubit register via swaps ---
def rotate4(circ: QuantumCircuit, qr: QuantumRegister, n: int):
    n %= bitsize 
    if n == 1:
        
        circ.swap(qr[0], qr[3])
        circ.swap(qr[2], qr[3])
        circ.swap(qr[1], qr[2])
    elif n == 2:
        
        circ.swap(qr[0], qr[2])
        circ.swap(qr[1], qr[3])
    elif n == 3:
        
        circ.swap(qr[0], qr[3])
        circ.swap(qr[0], qr[1])
        circ.swap(qr[1], qr[2])

def odd2_round(circ, qa, qb, qt):
    # make sure qt is all |0⟩ up front
    for i in range(bitsize):
        circ.reset(qt[i])

    # 1) a += b
    modulo_addition(circ, qb, qa)

    # 2) copy a→qt
    for i in range(bitsize):
        circ.cx(qa[i], qt[i])

    # 3) rotate-xor
    rotate4(circ, qa, 3)
    for i in range(bitsize):
        circ.cx(qt[i], qa[i])

    # 4) b += a
    modulo_addition(circ, qa, qb)


def build_toy_cipher(a_bits, b_bits, rounds: int = 5):
    qa = QuantumRegister(bitsize, 'a')
    qb = QuantumRegister(bitsize, 'b')
    qt = QuantumRegister(bitsize, 't')
    cr = ClassicalRegister(2 * bitsize, 'c')
    circ = QuantumCircuit(qa, qb, qt, cr)

    # initialize a and b from classical bits (LSB at index 0)
    for idx, bit in enumerate(reversed(a_bits)):
        if bit:
            circ.x(qa[idx])
    for idx, bit in enumerate(reversed(b_bits)):
        if bit:
            circ.x(qb[idx])

    # apply rounds
    for _ in range(rounds):
        odd2_round(circ, qa, qb, qt)

    # measure outputs: a in cr[0..3], b in cr[4..7]
    for i in range(bitsize): circ.measure(qa[i], cr[i])
    for i in range(bitsize): circ.measure(qb[i], cr[bitsize + i])

    return circ


if __name__ == '__main__':
    # plaintext 10011100 (b=0b1001 or 9, a=0b1100 or 12)
    
    key = 9  # random.randint(0, 2**bitsize - 1) a
    plaintext = 12 # random.randint(0, 2**bitsize - 1) b
    
    print(f"Key: {key:04b}, plaintext = {plaintext:04b}")

    state = f"{key:04b}{plaintext:04b}"
    print("8-bit initial state (MSB→LSB):", plaintext)
    
    a_bits = [int(bit) for bit in state[bitsize:]]
    b_bits = [int(bit) for bit in state[:bitsize]]

    circ = build_toy_cipher(a_bits, b_bits)

    sim = AerSimulator()
    job    = sim.run(circ, shots=1)
    result = job.result()
    counts = result.get_counts(circ)
    
    output_bin = max(counts, key=counts.get)
    output_int = int(output_bin, 2)
    
    # print the single outcome
    print(f"Ciphertext (binary MSB→LSB): {output_bin}")
    print(f"Ciphertext (decimal)   : {output_int}")


Key: 1001, plaintext = 1100
8-bit initial state (MSB→LSB): 12
Ciphertext (binary MSB→LSB): 10100000
Ciphertext (decimal)   : 160
